In [1]:
import requests
import pandas as pd
import os
from nypl_token import Token

# sets root path for directory creation and collection save
ROOT_PATH = os.path.dirname(os.path.abspath("__file__"))

# set API token, collection UUID, keyword for directory, and which image version you want to download, see below
token = Token.nypl_tok

uuid = '3ec86960-c6c7-012f-d945-58d385a7bc34'

collection_label = "fish"

# b - .jpeg center cropped thumbnail (100x100 pixels)
# f - .jpeg (140 pixels tall with variable width)
# t - .gif (150 pixels on the long side)
# r - .jpeg (300 pixels on the long side)
# w - .jpeg (760 pixels on the long side)
# q - .jpeg (1600 pixels on the long side)
# v - .jpeg (2560 pixels on the long side)
# g - .jpeg original dimensions

# NOTE: all but the high res files will be jpeg, if you plan to resize pull images in a larger format that what you will need
image_select = 'w'

# if not os.path.exists(ROOT_PATH):
os.mkdir(os.path.join(ROOT_PATH, collection_label))

# first call pulls down all subcoollection uuid's for parent and parses out relevant json
response = requests.get('http://api.repo.nypl.org/api/v1/collections/' + uuid + '?&per_page=500',
                        headers={'Authorization': 'Token token=' + token})

data = response.json()

df = pd.json_normalize(data)

df = df.explode('nyplAPI.response.collection', ignore_index=True)

df = pd.json_normalize(df['nyplAPI.response.collection'])

# df_next = df[['parentUUIDs', 'uuid']].copy()

# create sub-collection list for grabbing all items in sub-collections
sub_col = df['uuid'].tolist()

item_list = []

# second call grabs all items in sub-collection and parses download image links, can adjust to select high_res links if desired
for col in sub_col:
    response2 = requests.get('http://api.repo.nypl.org/api/v1/items/' + col + '?&per_page=500',
                             headers={'Authorization': 'Token token=' + token})

    item_data = response2.json()

    item_list.append(item_data)

df_item = pd.json_normalize(item_list)

df_item = df_item.explode('nyplAPI.response.capture', ignore_index=True)

df_item = pd.json_normalize(df_item['nyplAPI.response.capture'])

df_item_filt = df_item[['uuid', 'imageLinks.imageLink']].copy()

df_item_filt = df_item_filt.explode('imageLinks.imageLink', ignore_index=True)

# removes all but selected image type to download and creates list of download links
df_item_filt = df_item_filt[df_item_filt['imageLinks.imageLink'].str.contains('&t=' + image_select + '&download')]

img_list = df_item_filt['imageLinks.imageLink'].tolist()

# sets count to auto increment file name when saving, passes list of download links, saves each image in collection
count = 0

for img in img_list:
    response3 = requests.get(img, headers={'Authorization': 'Token token=' + token})

    count += 1

    # set image file type to match image_select type
    if response3.status_code == 200:
        with open(os.path.join(ROOT_PATH, collection_label) + "\image" + str(count) + ".jpg",
                  'wb') as f:
            f.write(response3.content)